In [1]:
import numpy as np
import scipy as sp
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz

In [2]:
with open('Training_Dataset.csv','rb') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    training_data=[]
    for row in readCSV:
        training_data.append(row)
training_data=training_data[1:]
training_data=np.asarray(training_data)
n=training_data.shape

In [3]:
catag=set(training_data[:,10])
catag=list(catag)
nc=len(catag)

In [4]:
data=[]
data_scaled=[]
supp=[]
elite=[]
credit=[]
nsupp=[]
nelite=[]
ncredit=[]
for i in range(0,nc):
    data.append([])
    data_scaled.append([])
    supp.append([])
    elite.append([])
    credit.append([])
    nsupp.append([])
    nelite.append([])
    ncredit.append([])
i=0
while i<n[0]:
    j=0
    while j<nc:
        if training_data[i,10]==catag[j]:
            data[j].append(training_data[i])
        j+=1
    i+=1

data=np.asarray(data)
    
nd=np.empty(nc,dtype='int')
for i in range(0,nc):
    nd[i]=len(data[i])
    supp[i]=np.zeros(nd[i],dtype='int')
    elite[i]=np.zeros(nd[i],dtype='int')
    credit[i]=np.zeros(nd[i],dtype='int')
    nsupp[i]=np.zeros(nd[i],dtype='int')
    nelite[i]=np.zeros(nd[i],dtype='int')
    ncredit[i]=np.zeros(nd[i],dtype='int')
    supp=np.asarray(supp)
    elite=np.asarray(elite)
    credit=np.asarray(credit)
    nsupp=np.asarray(nsupp)
    nelite=np.asarray(nelite)
    ncredit=np.asarray(ncredit)
    for j in range(0,nd[i]):
        x1=data[i][j][0:10]
        x2=data[i][j][11:-6]
        supp[i][j]=data[i][j][-3]
        elite[i][j]=data[i][j][-2]
        credit[i][j]=data[i][j][-1]
        nsupp[i][j]=data[i][j][-9]
        nelite[i][j]=data[i][j][-8]
        ncredit[i][j]=data[i][j][-7]
        data[i][j]=np.append(x1,x2)

In [5]:
scaler=MinMaxScaler()
for i in range(0,nc):
    scaler.fit(data[i])
    data_scaled[i]=scaler.transform(data[i])

In [6]:
svm_data=np.zeros([sum(nd),len(data_scaled[0][0])],dtype='float')
class_data=np.zeros(sum(nd),dtype='int')
count=0
for i in range(0,nc):
    for j in range(0,nd[i]):
        for k in range(0,len(data_scaled[0][0])):
            svm_data[count][k]=data_scaled[i][j][k]
        if elite[i][j]==1:
            class_data[count]=1
        elif supp[i][j]==1:
            class_data[count]=2
        elif credit[i][j]==1:
            class_data[count]=3
        else:
            class_data[count]=0
        count+=1

In [7]:
svm_data_=[]
class_data_=[]
count=0
for i in range(0,nc):
    svm_data_.append([])
    class_data_.append([])
    for j in range(0,nd[i]):
        svm_data_[i].append([])
        class_data_[i].append([])
        for k in range(0,len(data_scaled[0][0])):
            svm_data_[i][j].append(data_scaled[i][j][k])
        if elite[i][j]==1:
            class_data_[i][j]=1
        elif supp[i][j]==1:
            class_data_[i][j]=2
        elif credit[i][j]==1:
            class_data_[i][j]=3
        else:
            class_data_[i][j]=0
        count+=1

In [8]:
svm_data_=np.asarray(svm_data_)
class_data_=np.asarray(class_data_)

In [9]:
clf_svm=np.empty(nc,dtype='object')
clf_dt=np.empty(nc,dtype='object')
score=np.zeros(nc,dtype='float')
score_svm=np.zeros(nc,dtype='float')
for i in range(0,nc):
    X_train, X_test, y_train, y_test = train_test_split(svm_data_[i],class_data_[i])
    clf_dt[i]=DecisionTreeClassifier().fit(X_train,y_train)
    clf_svm[i]=SVC().fit(X_train,y_train)
    score[i]=clf_dt[i].score(X_test,y_test)
    score_svm[i]=clf_svm[i].score(X_test,y_test)

In [10]:
export_graphviz(clf_dt[3])

E:\graphlab\Anaconda2\lib\site-packages\sklearn\tree\export.py:399: DeprecationWarning: out_file can be set to None starting from 0.18. This will be the default in 0.20.
  DeprecationWarning)


In [11]:
with open('Leaderboard_Dataset.csv','r') as csvfile1:
    readCSV1 = csv.reader(csvfile1, delimiter=',')
    testing_data=[]
    for row1 in readCSV1:
        testing_data.append(row1)
testing_data=testing_data[1:]
testing_data=np.asarray(testing_data)
n=testing_data.shape
cmkey=testing_data[:,0]

In [12]:
test_catag=testing_data[:,11]
test_catag_index=np.zeros(n[0],dtype='int')
for j in range(0,n[0]):
    for i in range(0,nc):
        if test_catag[j]==catag[i]:
            test_catag_index[j]=i

In [13]:
x1=testing_data[:,1:11]
x2=testing_data[:,12:]
testing_data=np.append(x1,x2)

In [14]:
testing_data=np.empty([n[0],len(x1[0])+len(x2[0])],dtype='float')
for i in range(0,n[0]):
    for j in range(0,len(x1[0])):
        testing_data[i][j]=x1[i][j]
    for j in range(len(x1[0])+1,len(x2[0])):
        testing_data[i][j]=x2[i][j]

In [15]:
scaler=MinMaxScaler()
testing_data_scaled=np.empty(n[0],dtype='object')
out=np.empty(n[0],dtype='object')
out_svm=np.empty(n[0],dtype='object')
for j in range(0,n[0]):
    i=test_catag_index[j]
    scaler.fit(data[i])
    testing_data_scaled[j]=scaler.transform([testing_data[j]])
    out[j]=[cmkey[j],clf_dt[i].predict(testing_data_scaled[j]),score[i]]
    out_svm[j]=[cmkey[j],clf_svm[i].predict(testing_data_scaled[j]),score_svm[i]]

In [16]:
class_key=['Elite','Supp','Credit','none']
out_array=np.zeros([n[0],len(out[0])],dtype='float')
out_array_svm=np.zeros([n[0],len(out[0])],dtype='float')
f=open('Leaderboard_output_dt.csv','w')
f_svm=open('Leaderboard_output_svm.csv','w')
for i in range(0,n[0]):
    for j in range(0,len(out[0])):
        out_array[i][j]=out[i][j]
        out_array_svm[i][j]=out_svm[i][j]
    f.write('%d,'%out_array[i][0])
    f.write(class_key[int(out_array[i][1])])
    f.write(',%f\n'%out_array[i][2])
    f_svm.write('%d,'%out_array_svm[i][0])
    f_svm.write(class_key[int(out_array_svm[i][1])])
    f_svm.write(',%f\n'%out_array_svm[i][2])
f.close()
f_svm.close()
np.savetxt("Leaderboard_output_dt_np.csv",out_array,delimiter=",")
np.savetxt("Leaderboard_output_svm_np.csv",out_array,delimiter=",")

In [17]:
# from sklearn.linear_model import LogisticRegression
# clf_e=np.empty(nc,dtype='object')
# clf_s=np.empty(nc,dtype='object')
# clf_c=np.empty(nc,dtype='object')
# e=np.zeros(nc,dtype='int')
# s=np.zeros(nc,dtype='int')
# c=np.zeros(nc,dtype='int')
# for i in range(0,nc):
#     e[i]=s[i]=c[i]=0
#     for j in range(0,nd[i]):
#         if elite[i][j]==1:
#             e[i]+=1
#         if supp[i][j]==1:
#             s[i]+=1
#         if credit[i][j]==1:
#             c[i]+=1
#     if e[i]==0:
#         data_scaled[i]=np.append(data_scaled[i],data_scaled[i][-1])
#         elite[i]=np.append(elite[i],1)
#         supp[i]=np.append(supp[i],0)
#         credit[i]=np.append(credit[i],0)
#     if s[i]==0:
#         data_scaled[i]=np.append(data_scaled[i],data_scaled[i][-1])
#         elite[i]=np.append(elite[i],0)
#         supp[i]=np.append(supp[i],1)
#         credit[i]=np.append(credit[i],0)
#     if c[i]==0:
#         data_scaled[i]=np.append(data_scaled[i],data_scaled[i][-1])
#         elite[i]=np.append(elite[i],0)
#         supp[i]=np.append(supp[i],0)
#         credit[i]=np.append(credit[i],1)
        
#     clf_e[i]=LogisticRegression(C=100).fit(data_scaled[i],elite[i])
#     clf_s[i]=LogisticRegression(C=100).fit(data_scaled[i],supp[i])
#     clf_c[i]=LogisticRegression(C=100).fit(data_scaled[i],credit[i])